### **ii.	Разработка и тестирование синтаксического анализатора для автоматизированной обработки документов форматов .doc, .pdf, .docx, djvu**

#### **`.docx`**

Для извлечения текста из .docx воспользуемся библиотекой [`python-docx`](https://python-docx.readthedocs.io/)

In [1]:
# для решения проблемы видимости текста с гиперссылками

from docx.text.paragraph import Paragraph
import re

Paragraph.text = property(lambda self: GetParagraphText(self))
def GetParagraphText(paragraph):

    def GetTag(element):
        return "%s:%s" % (element.prefix, re.match("{.*}(.*)", element.tag).group(1))

    text = ''
    runCount = 0
    for child in paragraph._p:
        tag = GetTag(child)
        if tag == "w:r":
            text += paragraph.runs[runCount].text
            runCount += 1
        if tag == "w:hyperlink":
            for subChild in child:
                if GetTag(subChild) == "w:r":
                    text += subChild.text
    return text

In [2]:
#!pip install python-docx
# Можем посмотреть на общую информацию

import docx

from docx import Document
document = Document('example.docx')

paragraphs = document.paragraphs
print('Общее количество различных параграфов:', len(paragraphs))

header = paragraphs[0]
print('Заголовок первого уровня:', header.text)

paragraph1 = paragraphs[1]
print('Текст первого абзаца:', paragraph1.text.strip())

Общее количество различных параграфов: 111
Заголовок первого уровня: Лучшие сериалы — 2022: выбор Кинопоиска
Текст первого абзаца: «Очень странные дела», 4-й сезон


Получим весь текст документа (без таблиц) - объект `paragraph`

In [3]:
print('Весь текст документа:')
text = []
for paragraph in document.paragraphs:
    text.append(paragraph.text)
print('\n'.join(text)) # cимвол новой строки 

Весь текст документа:
Лучшие сериалы — 2022: выбор Кинопоиска
«Очень странные дела», 4-й сезон
Даже в год «Дома Дракона» и «Колец власти» работа братьев Даффер не спешит покидать трон главного сериального блокбастера планеты. К четвертому сезону, который стал самым популярным англоязычным шоу в истории Netflix, «Очень странные дела» разрослись до поистине исполинских масштабов. Как в самом буквальном смысле (13 часов действия, в два раза длиннее первого сезона; финал на беспрецедентные два с половиной часа), так и в содержательном (история о приключениях группы школьников в провинциальном городке окончательно приобрела размах эпоса). Героев стало так много, что их пришлось развести по параллельным квестам, как в «Игре престолов». К уютному Хокинсу присоединились равнодушная Калифорния и совсем уж недружелюбная Камчатка. Ставки для человечества драматически повысились: финальные сцены напоминают уже что-то не из Спилберга даже, а из Роланда Эммериха.
Среди неоспоримых удач сезона — перв

Получим весь текст из таблиц - объект `table`

In [4]:
for table in document.tables:
    for row in table.rows:
        for cell in row.cells:
            print(cell.text)

Название
Рейтинг
Год
Очень странные дела
8.4
2016 – 2024
Дом Дракона
8.2
2022 – 2022
Белый лотос
7.5
2021
Нулевой пациент
8.4
2022 – 2022


Можем извлечь все фрагменты, написанные курсивом - `run.italic`

In [5]:
for paragraph in document.paragraphs:
    for run in paragraph.runs:
        if run.italic:
            print(run.text) 

Режиссер: Мэтт 
Даффер
, Росс 
Даффер
, Шон Леви
В главных ролях: Вайнона Райдер, Дэвид Харбор, Финн 
Вулфхард
Режиссер: Клер 
Килнер
, Мигель 
Сапочник
, 
Грег
 
Яйтанс
В главных ролях: Пэдди 
Консидайн
, Мэтт Смит, Эмма 
Д’Арси
Режиссер: Майк Уайт
В главных ролях: 
Мюррэй
 
Бартлетт
, Конни 
Бриттон
, Дженнифер 
Кулидж
Режиссер: Сергей Трофимов, Евгений Стычкин
В главных ролях: Аскар Ильясов, Никита Ефремов, Елизавета 
Шакира
Режиссер: Сэм Левинсон, 
Пиппа
 
Бьянко
, Августин 
Фриззелл
В главных ролях: 
Зендея
, Хантер Шафер, Джейкоб 
Элорди
Режиссер: Джеймс Ганн, Брэд Андерсон, Джоди Хилл
В главных ролях: Джон Сина, Даниэль Брукс, Фредди Строма
Режиссер: 
Нэйтан
 
Филдер
В главных ролях: 
Нэйтан
 
Филдер
, 
Journey
 Baker, 
Anna
 
Lamadrid
Режиссер: Тоби 
Хэйнс
, Бенджамин Карон, Сюзанна Уайт
В главных ролях: Диего Луна, Кайл 
Соллер
, 
Стеллан
 
Скарсгард
Режиссер: Винс 
Гиллиган
, Томас 
Шнауз
, Питер Гулд
В главных ролях: Боб 
Оденкёрк
, Джонатан Бэнкс, Рэй 
Сихорн
Режиссер: Крис

Можем извлечь подчеркнутый текст - `run.underline`

In [6]:
for paragraph in document.paragraphs:
    for run in paragraph.runs:
        if run.underline:
            print(run.text) 

Джорджем Мартином
Джонатан 
Франзен
Евгения Стычкина


Полужирный текст - `run.bold`

In [7]:
for paragraph in document.paragraphs:
    for run in paragraph.runs:
        if run.bold:
            print(run.text)

Лучшие сериалы — 2022: выбор 
Кинопоиска
«Белый лотос», 2-й сезон
«Эйфория», 2-й сезон
«Репетиция»
«
Андор
»
«Лучше звоните Солу», 6-й сезон
«Медведь»
«Барри», 3-й сезон
«Властелин колец: Кольца власти»
«Полиция Токио»
«Разделение»
«Атланта», 3-й и 4-й сезоны
«Ирма 
Веп
»
«Капельник»
«Медленные лошади», 1-й и 2-й сезоны
«Убийства в одном здании», 2-й сезон
«Конец света»
«Лунный рыцарь»
«Дорога в тысячу ли»
«Почка»
«Черная весна»
«ЮЗЗЗ»


#### **`.pdf`**

Для извлечения всего текста из файла, воспользуемся библиотекой [PDFMiner.six.](https://github.com/pdfminer/pdfminer.six) - для Python 3. Включает поддержку различных языков (китайский, японский и корейский) и различных шрифтов (Type1, Type3 и CID) ([тутор](https://python-scripts.com/exporting-data-from-pdf)).

In [8]:
#!pip install pdfminer

import io
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
 
# создадим функцию-генератор для извлечения текста каждой страницы
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text
 
if __name__ == '__main__':
    print(extract_text_from_pdf('example.pdf'))

Лучшие сериалы — 2022: выбор Кинопоиска «Очень странные дела», 4-й сезон Даже в год «Дома Дракона» и «Колец власти» работа братьев Даффер не спешит покидать трон главного сериального блокбастера планеты. К четвертому сезону, который стал самым популярным англоязычным шоу в истории Netflix, «Очень странные дела» разрослись до поистине исполинских масштабов. Как в самом буквальном смысле (13 часов действия, в два раза длиннее первого сезона; финал на беспрецедентные два с половиной часа), так и в содержательном (история о приключениях группы школьников в провинциальном городке окончательно приобрела размах эпоса). Героев стало так много, что их пришлось развести по параллельным квестам, как в «Игре престолов». К уютному Хокинсу присоединились равнодушная Калифорния и совсем уж недружелюбная Камчатка. Ставки для человечества драматически повысились: финальные сцены напоминают уже что-то не из Спилберга даже, а из Роланда Эммериха. Среди неоспоримых удач сезона — первый (и, вероятно, уже п

Еще один способ извлечения текста

In [9]:
from pdfminer.high_level import extract_text

# Extract text from a pdf.
text = extract_text('example.pdf')
print(text)

Лучшие сериалы — 2022: выбор Кинопоиска 

«Очень странные дела», 4-й сезон 

Даже в год «Дома Дракона» и «Колец власти» работа братьев Даффер не спешит покидать 
трон главного сериального блокбастера планеты. К четвертому сезону, который стал самым 
популярным англоязычным шоу в истории Netflix, «Очень странные дела» разрослись до 
поистине исполинских масштабов. Как в самом буквальном смысле (13 часов действия, в 
два раза длиннее первого сезона; финал на беспрецедентные два с половиной часа), так и 
в  содержательном  (история  о  приключениях  группы  школьников  в  провинциальном 
городке окончательно приобрела размах эпоса). Героев стало так много, что их пришлось 
развести  по  параллельным  квестам,  как  в  «Игре  престолов».  К  уютному  Хокинсу 
присоединились равнодушная Калифорния и совсем уж недружелюбная Камчатка. Ставки 
для человечества драматически повысились: финальные сцены напоминают уже что-то не 
из Спилберга даже, а из Роланда Эммериха. 

Среди  неоспоримых  удач

Тут можем вывести кол-во абзацев, воспользовавшись регулярными выражениями

In [10]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import re

password = "R@m@_f0rtu9e$"
# Open a PDF document
fp = open('example.pdf', 'rb')
parser = PDFParser(fp)
document = PDFDocument(parser, password)

# Get the outlines of the document
outlines = document.get_outlines()
for (level,title,dest,a,se) in outlines:
    print (title)

Лучшие сериалы — 2022: выбор Кинопоиска
«Очень странные дела», 4-й сезон
«Дом Дракона»
«Белый лотос», 2-й сезон
«Нулевой пациент»
«Эйфория», 2-й сезон
«Миротворец»
«Репетиция»
«Андор»
«Лучше звоните Солу», 6-й сезон
«Медведь»
«Барри», 3-й сезон
«Властелин колец: Кольца власти»
«Полиция Токио»
«Разделение»
«Атланта», 3-й и 4-й сезоны
«Ирма Веп»
«Капельник»
«Медленные лошади», 1-й и 2-й сезоны
«Убийства в одном здании», 2-й сезон
«Конец света»
«Лунный рыцарь»
«Дорога в тысячу ли»
«Почка»
«Черная весна»
«ЮЗЗЗ»


Конвертация в txt

In [11]:
from pdfminer.layout import LAParams

from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

a = convert_pdf_to_txt('example.pdf')
a
# len(a) кол-во символов можем посчитать

'Лучшие сериалы — 2022: выбор Кинопоиска \n\n«Очень странные дела», 4-й сезон \n\nДаже в год «Дома Дракона» и «Колец власти» работа братьев Даффер не спешит покидать \nтрон главного сериального блокбастера планеты. К четвертому сезону, который стал самым \nпопулярным англоязычным шоу в истории Netflix, «Очень странные дела» разрослись до \nпоистине исполинских масштабов. Как в самом буквальном смысле (13 часов действия, в \nдва раза длиннее первого сезона; финал на беспрецедентные два с половиной часа), так и \nв  содержательном  (история  о  приключениях  группы  школьников  в  провинциальном \nгородке окончательно приобрела размах эпоса). Героев стало так много, что их пришлось \nразвести  по  параллельным  квестам,  как  в  «Игре  престолов».  К  уютному  Хокинсу \nприсоединились равнодушная Калифорния и совсем уж недружелюбная Камчатка. Ставки \nдля человечества драматически повысились: финальные сцены напоминают уже что-то не \nиз Спилберга даже, а из Роланда Эммериха. \n\nСреди  

Извлекаем таблицу в формат датафрейма при помощи библиотеки [`camelot`](https://camelot-py.readthedocs.io/en/master/) - ([тутор](https://newtechaudit.ru/izvlechenie-tablicz-iz-pdf-s-pomoshhyu-camelot/)).

In [12]:
# Необходимо установить зависимости

# !pip install pdf2image
# !pip install python-dateutil
# !apt-get install poppler-utils 

# !git clone https://www.github.com/h2oai/camelot
# !cd camelot
# !pip install ".[base]"

import pandas as pd
import camelot 
# Так импортируется PyMuPDF 
import sys, fitz

class ConversionBackend(object):
    def convert(self, pdf_path, png_path):
        # Открываем документ
        doc = fitz.open(pdf_path) 
        for page in doc.pages():
            # Переводим страницу в картинку
            pix = page.get_pixmap()  
            # Сохраняем
            pix.save(png_path)

file = 'example.pdf'
tables = camelot.read_pdf(file, 
                          backend=ConversionBackend(), 
                          strip_text='\n', 
                          line_scale=40, 
                          pages='all',
                          copy_text=['h'],)

tables[0].df

,0,1,2
0,Название,Рейтинг,Год
1,Очень странные дела,8.4,2016 – 2024
2,Дом Дракона,8.2,2022 – 2022
3,Белый лотос,7.5,2021
4,Нулевой пациент,8.4,2022 – 2022


Извлекаем гиперссылки при помощи библиотеки `PyPDF2`

In [13]:
import PyPDF2
PDFFile = open("example.pdf",'rb')

PDF = PyPDF2.PdfFileReader(PDFFile)
pages = PDF.getNumPages()
key = '/Annots'
uri = '/URI'
ank = '/A'

for page in range(pages):
    print("Current Page: {}".format(page))
    pageSliced = PDF.getPage(page)
    pageObject = pageSliced.getObject()
    if key in pageObject.keys():
        ann = pageObject[key]
        for a in ann:
            u = a.getObject()
            if uri in u[ank].keys():
                print('Гиперссылка:', u[ank][uri])

Current Page: 0
Гиперссылка: https://ru.wikipedia.org/wiki/%D0%9E%D1%87%D0%B5%D0%BD%D1%8C_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5_%D0%B4%D0%B5%D0%BB%D0%B0_(4-%D0%B9_%D1%81%D0%B5%D0%B7%D0%BE%D0%BD)
Гиперссылка: https://ru.wikipedia.org/wiki/%D0%94%D0%BE%D0%BC_%D0%94%D1%80%D0%B0%D0%BA%D0%BE%D0%BD%D0%B0
Current Page: 1
Гиперссылка: https://ru.wikipedia.org/wiki/%D0%9D%D1%83%D0%BB%D0%B5%D0%B2%D0%BE%D0%B9_%D0%BF%D0%B0%D1%86%D0%B8%D0%B5%D0%BD%D1%82_(%D1%81%D0%B5%D1%80%D0%B8%D0%B0%D0%BB)
Current Page: 2
Current Page: 3
Гиперссылка: https://ru.wikipedia.org/wiki/%D0%9C%D0%B8%D1%80%D0%BE%D1%82%D0%B2%D0%BE%D1%80%D0%B5%D1%86_(%D1%81%D0%B5%D1%80%D0%B8%D0%B0%D0%BB)
Current Page: 4
Current Page: 5
Current Page: 6
Current Page: 7
Current Page: 8
Current Page: 9
Current Page: 10
Гиперссылка: https://www.kinopoisk.ru/film/915196/


#### **`.doc`**

Для извлечения текста воспользуемся модулем `textract` - позволяет читать файлы любого формата

In [14]:
# !pip install textract

import textract
text = textract.process('example.doc')
text.decode('utf-8')  # # преобразует строку байтов в строку

'\n\n                   Лучшие сериалы — 2022: выбор Кинопоиска\n\n\n«Очень странные дела», 4-й сезон\n\nДаже в год «Дома Дракона» и «Колец власти» работа братьев Даффер  не  спешит\nпокидать  трон  главного  сериального  блокбастера  планеты.  К   четвертому\nсезону, который стал самым популярным англоязычным шоу  в  истории  Netflix,\n«Очень странные дела» разрослись до поистине исполинских  масштабов.  Как  в\nсамом буквальном смысле (13 часов  действия,  в  два  раза  длиннее  первого\nсезона;  финал  на  беспрецедентные  два  с  половиной  часа),   так   и   в\nсодержательном (история о приключениях группы  школьников  в  провинциальном\nгородке окончательно приобрела размах эпоса). Героев стало  так  много,  что\nих пришлось развести по параллельным квестам,  как  в  «Игре  престолов».  К\nуютному  Хокинсу  присоединились  равнодушная   Калифорния   и   совсем   уж\nнедружелюбная Камчатка. Ставки  для  человечества  драматически  повысились:\nфинальные сцены напоминают уже что-то

Конвертируем .doc в .docx 

In [15]:
# ?????

import aspose.words as aw

doc = aw.Document("example.doc")
doc.save("Output.docx")

# на этом этапе возникает ошибка 'Canceled future for execute_request message before replies were done' kernel Python 3.10.6 died

: 

: 

#### **`.djvu`**

Можем попробовать конвертировать .djvu в .pdf через linux и работать с pdf

In [ ]:
# !sudo apt install djvulibre-bin
# !ddjvu -format=pdf prestupleniye_i_nakazaniye.{djvu,pdf} 